### This Notebook finds the values in a single county from a gridded data. A sample of 100 latitude and longitude pair is taken from the shapefile of a county. The values of the variables from the netcdf are then interpolated from the grid to this set of latitude and longitude.

In [1]:
## conda environment Weather_Prediction

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr

In [2]:
data = xr.open_dataset("data.nc") ## data with one month average of the variables
data

<xarray.Dataset> Size: 7MB
Dimensions:        (valid_time: 12, latitude: 35, longitude: 81)
Coordinates:
  * valid_time     (valid_time) datetime64[ns] 96B 2010-01-01 ... 2010-12-01
  * latitude       (latitude) float64 280B 49.5 48.75 48.0 ... 25.5 24.75 24.0
  * longitude      (longitude) float64 648B -126.0 -125.2 ... -66.75 -66.0
Data variables: (12/48)
    d2m            (valid_time, latitude, longitude) float32 136kB ...
    t2m            (valid_time, latitude, longitude) float32 136kB ...
    bcaod550       (valid_time, latitude, longitude) float32 136kB ...
    chnk           (valid_time, latitude, longitude) float32 136kB ...
    duaod550       (valid_time, latitude, longitude) float32 136kB ...
    lai_hv         (valid_time, latitude, longitude) float32 136kB ...
    ...             ...
    aermssomhphob  (valid_time, latitude, longitude) float32 136kB ...
    aermsssss      (valid_time, latitude, longitude) float32 136kB ...
    aermssssm      (valid_time, latitude, longitude) float32 136kB ...
    aermssssl      (valid_time, latitude, longitude) float32 136kB ...
    aermsssu       (valid_time, latitude, longitude) float32 136kB ...
    aermssso2      (valid_time, latitude, longitude) float32 136kB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-10-10T00:09 GRIB to CDM+CF via cfgrib-0.9.1...

In [3]:
## resample the data to 1 year average

year_avg = data.resample(valid_time='1YE').mean(dim='valid_time')
year_avg

<xarray.Dataset> Size: 545kB
Dimensions:        (valid_time: 1, latitude: 35, longitude: 81)
Coordinates:
  * latitude       (latitude) float64 280B 49.5 48.75 48.0 ... 25.5 24.75 24.0
  * longitude      (longitude) float64 648B -126.0 -125.2 ... -66.75 -66.0
  * valid_time     (valid_time) datetime64[ns] 8B 2010-12-31
Data variables: (12/48)
    d2m            (valid_time, latitude, longitude) float32 11kB 278.3 ... 2...
    t2m            (valid_time, latitude, longitude) float32 11kB 281.6 ... 2...
    bcaod550       (valid_time, latitude, longitude) float32 11kB 0.006512 .....
    chnk           (valid_time, latitude, longitude) float32 11kB 0.01415 ......
    duaod550       (valid_time, latitude, longitude) float32 11kB 0.001253 .....
    lai_hv         (valid_time, latitude, longitude) float32 11kB 3.752 ... 0.0
    ...             ...
    aermssomhphob  (valid_time, latitude, longitude) float32 11kB 9.168e-06 ....
    aermsssss      (valid_time, latitude, longitude) float32 11kB 6.539e-07 ....
    aermssssm      (valid_time, latitude, longitude) float32 11kB 4.983e-05 ....
    aermssssl      (valid_time, latitude, longitude) float32 11kB 1.505e-05 ....
    aermsssu       (valid_time, latitude, longitude) float32 11kB 2.849e-06 ....
    aermssso2      (valid_time, latitude, longitude) float32 11kB 6.057e-07 ....
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-10-10T00:09 GRIB to CDM+CF via cfgrib-0.9.1...

In [4]:
## delete variables that have NaN in them

vars_with_nan = [var for var in year_avg if year_avg[var].isnull().any()]
year_avg= year_avg.drop_vars(vars_with_nan)
year_avg

<xarray.Dataset> Size: 523kB
Dimensions:        (valid_time: 1, latitude: 35, longitude: 81)
Coordinates:
  * latitude       (latitude) float64 280B 49.5 48.75 48.0 ... 25.5 24.75 24.0
  * longitude      (longitude) float64 648B -126.0 -125.2 ... -66.75 -66.0
  * valid_time     (valid_time) datetime64[ns] 8B 2010-12-31
Data variables: (12/46)
    d2m            (valid_time, latitude, longitude) float32 11kB 278.3 ... 2...
    t2m            (valid_time, latitude, longitude) float32 11kB 281.6 ... 2...
    bcaod550       (valid_time, latitude, longitude) float32 11kB 0.006512 .....
    chnk           (valid_time, latitude, longitude) float32 11kB 0.01415 ......
    duaod550       (valid_time, latitude, longitude) float32 11kB 0.001253 .....
    lai_hv         (valid_time, latitude, longitude) float32 11kB 3.752 ... 0.0
    ...             ...
    aermssomhphob  (valid_time, latitude, longitude) float32 11kB 9.168e-06 ....
    aermsssss      (valid_time, latitude, longitude) float32 11kB 6.539e-07 ....
    aermssssm      (valid_time, latitude, longitude) float32 11kB 4.983e-05 ....
    aermssssl      (valid_time, latitude, longitude) float32 11kB 1.505e-05 ....
    aermsssu       (valid_time, latitude, longitude) float32 11kB 2.849e-06 ....
    aermssso2      (valid_time, latitude, longitude) float32 11kB 6.057e-07 ....
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-10-10T00:09 GRIB to CDM+CF via cfgrib-0.9.1...

In [5]:
## note that even though the time dimesion, essentially shows 2010-12-31,
## what it actually means is yearly average
year_avg.to_dataframe()

d2m         t2m  bcaod550      chnk  \
valid_time latitude longitude                                               
2010-12-31 49.5     -126.00    278.348907  281.556824  0.006512  0.014148   
                    -125.25    279.088837  282.437836  0.006916  0.013934   
                    -124.50    279.470001  282.923370  0.007213  0.013523   
                    -123.75    279.041290  282.562347  0.007478  0.013581   
                    -123.00    277.851837  281.525085  0.008017  0.014813   
...                                   ...         ...       ...       ...   
           24.0     -69.00     293.523407  297.542999  0.004872  0.012293   
                    -68.25     293.495728  297.488953  0.004856  0.012303   
                    -67.50     293.465454  297.455261  0.004833  0.012302   
                    -66.75     293.443970  297.427094  0.004824  0.012318   
                    -66.00     293.438751  297.397156  0.004825  0.012332   

                               duaod550    lai_hv    lai_lv            msl  \
valid_time latitude longitude                                                
2010-12-31 49.5     -126.00    0.001253  3.752065  2.928701  101390.726562   
                    -125.25    0.001246  4.228424  1.811178  101402.250000   
                    -124.50    0.001238  2.194234  0.692861  101427.570312   
                    -123.75    0.001204  0.905955  0.624898  101459.429688   
                    -123.00    0.001140  3.628265  2.724324  101485.656250   
...                                 ...       ...       ...            ...   
           24.0     -69.00     0.024328  0.000000  0.000000  101571.789062   
                    -68.25     0.024477  0.000000  0.000000  101578.000000   
                    -67.50     0.024507  0.000000  0.000000  101583.164062   
                    -66.75     0.024585  0.000000  0.000000  101590.195312   
                    -66.00     0.024893  0.000000  0.000000  101595.625000   

                               omaod550         pm2p5  ...     aermssdul  \
valid_time latitude longitude                          ...                 
2010-12-31 49.5     -126.00    0.055563  5.288607e-09  ...  5.697568e-07   
                    -125.25    0.060980  6.618710e-09  ...  5.552510e-07   
                    -124.50    0.063776  7.792362e-09  ...  5.443322e-07   
                    -123.75    0.067136  1.080445e-08  ...  5.231616e-07   
                    -123.00    0.076543  1.931933e-08  ...  4.871432e-07   
...                                 ...           ...  ...           ...   
           24.0     -69.00     0.046011  9.519893e-09  ...  1.086019e-05   
                    -68.25     0.045661  9.399162e-09  ...  1.102058e-05   
                    -67.50     0.045540  9.288230e-09  ...  1.103588e-05   
                    -66.75     0.045691  9.213097e-09  ...  1.106749e-05   
                    -66.00     0.045851  9.209978e-09  ...  1.116341e-05   

                               aermssbchphil  aermssomhphil  aermssbchphob  \
valid_time latitude longitude                                                
2010-12-31 49.5     -126.00     1.359443e-07   2.107467e-06   5.559153e-07   
                    -125.25     1.506114e-07   2.379295e-06   5.843853e-07   
                    -124.50     1.616488e-07   2.564180e-06   6.047914e-07   
                    -123.75     1.798799e-07   2.806439e-06   6.146706e-07   
                    -123.00     2.252995e-07   3.389544e-06   6.264335e-07   
...                                      ...            ...            ...   
           24.0     -69.00      3.103107e-08   5.009322e-07   4.866120e-07   
                    -68.25      3.011268e-08   4.867323e-07   4.857583e-07   
                    -67.50      2.886006e-08   4.733052e-07   4.846137e-07   
                    -66.75      2.775320e-08   4.601205e-07   4.847639e-07   
                    -66.00      2.699588e-08   4.502798e-07   4.856791e-07   

                 

In [6]:
## Lets load the county shapefile

SHAPE_PATH = os.path.join("County_shapefile",'gz_2010_us_050_00_500k.shp')
county_gdf = gpd.read_file(SHAPE_PATH)

county_gdf.head()

,GEO_ID,STATE,COUNTY,NAME,LSAD,CENSUSAREA,geometry
0,0500000US01029,01,029,Cleburne,County,560.100,"POLYGON ((-85.38872 33.91304, -85.38088 33.873..."
1,0500000US01031,01,031,Coffee,County,678.972,"POLYGON ((-86.03044 31.61894, -86.00408 31.619..."
2,0500000US01037,01,037,Coosa,County,650.926,"POLYGON ((-86.00928 33.10164, -86.00917 33.090..."
3,0500000US01039,01,039,Covington,County,1030.456,"POLYGON ((-86.34851 30.99434, -86.35023 30.994..."
4,0500000US01041,01,041,Crenshaw,County,608.840,"POLYGON ((-86.14699 31.68045, -86.14711 31.663..."


In [7]:
geometry_column=county_gdf['geometry'] ## get the geometry column
geometry_column

0       POLYGON ((-85.38872 33.91304, -85.38088 33.873...
1       POLYGON ((-86.03044 31.61894, -86.00408 31.619...
2       POLYGON ((-86.00928 33.10164, -86.00917 33.090...
3       POLYGON ((-86.34851 30.99434, -86.35023 30.994...
4       POLYGON ((-86.14699 31.68045, -86.14711 31.663...
                              ...                        
3216    POLYGON ((-66.90748 18.25314, -66.90739 18.253...
3217    POLYGON ((-66.37968 17.94398, -66.38029 17.943...
3218    MULTIPOLYGON (((-66.13957 18.46232, -66.13956 ...
3219    POLYGON ((-66.02917 18.37590, -66.02828 18.376...
3220    POLYGON ((-66.85229 17.95500, -66.85280 17.955...
Name: geometry, Length: 3221, dtype: geometry

In [8]:
type(geometry_column)

geopandas.geoseries.GeoSeries

In [9]:
lat_lon=geometry_column.get_coordinates(index_parts=True) ## get lat and lon from the geometry
lat_lon

x          y
0    0   -85.388717  33.913044
     1   -85.380885  33.873508
     2   -85.379455  33.866291
     3   -85.377426  33.856047
     4   -85.376403  33.850656
...             ...        ...
3220 202 -66.833718  17.989763
     203 -66.835282  17.988274
     204 -66.835429  17.986323
     205 -66.836682  17.965971
     206 -66.852288  17.955004

[837022 rows x 2 columns]

In [10]:
type(lat_lon)

pandas.core.frame.DataFrame

## Find value of the variable in the county with index 0 in the shape file

In [11]:
longitude= lat_lon.loc[(0), 'x']  # get the longitude values
longitude=longitude.sample(n=10)  ## extract a random sample of 10 longitude
longitude

33    -85.352576
62    -85.851890
17    -85.313999
49    -85.680346
6     -85.361844
9     -85.355252
21    -85.314852
109   -85.495289
71    -85.724953
103   -85.601858
Name: x, dtype: float64

In [12]:
latitude= lat_lon.loc[(0), 'y']   # get the latitude values
latitude=latitude.sample(n=10)    ## extract a random sample of 10 latitude
latitude

63     33.499690
88     33.773339
115    33.927068
77     33.625105
92     33.787755
68     33.600002
75     33.612909
52     33.496862
32     33.498866
26     33.494700
Name: y, dtype: float64

In [13]:
lat_list=latitude.tolist()
lat_list

[33.49969,
 33.773339,
 33.927068064286296,
 33.625105,
 33.787755,
 33.600001999999996,
 33.612909,
 33.496862,
 33.498866,
 33.4947]

In [14]:
lon_list=longitude.tolist()
lon_list

[-85.352576,
 -85.85189,
 -85.313999,
 -85.680346,
 -85.36184399999999,
 -85.35525221574429,
 -85.314852,
 -85.495289,
 -85.724953,
 -85.601858]

In [15]:
## find the corresponding values of the variables in the finer grid

year_avg_finer= year_avg.interp(longitude=lon_list, latitude=lat_list)
year_avg_finer

<xarray.Dataset> Size: 37kB
Dimensions:        (valid_time: 1, latitude: 10, longitude: 10)
Coordinates:
  * valid_time     (valid_time) datetime64[ns] 8B 2010-12-31
  * longitude      (longitude) float64 80B -85.35 -85.85 -85.31 ... -85.72 -85.6
  * latitude       (latitude) float64 80B 33.5 33.77 33.93 ... 33.5 33.5 33.49
Data variables: (12/46)
    d2m            (valid_time, latitude, longitude) float64 800B 282.6 ... 2...
    t2m            (valid_time, latitude, longitude) float64 800B 289.8 ... 2...
    bcaod550       (valid_time, latitude, longitude) float64 800B 0.006262 .....
    chnk           (valid_time, latitude, longitude) float64 800B 0.018 ... 0...
    duaod550       (valid_time, latitude, longitude) float64 800B 0.007976 .....
    lai_hv         (valid_time, latitude, longitude) float64 800B 3.651 ... 3...
    ...             ...
    aermssomhphob  (valid_time, latitude, longitude) float64 800B 1.286e-05 ....
    aermsssss      (valid_time, latitude, longitude) float64 800B 2.016e-07 ....
    aermssssm      (valid_time, latitude, longitude) float64 800B 1.333e-05 ....
    aermssssl      (valid_time, latitude, longitude) float64 800B 7.536e-07 ....
    aermsssu       (valid_time, latitude, longitude) float64 800B 8.705e-06 ....
    aermssso2      (valid_time, latitude, longitude) float64 800B 3.842e-06 ....
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-10-10T00:09 GRIB to CDM+CF via cfgrib-0.9.1...

### Note that xarray will provide a 10 X 10 = 100 pair of latitude and longitude, that is, interpolated at 100 points.

In [16]:
year_avg_finer.to_dataframe()

d2m         t2m  bcaod550   chnk  \
valid_time latitude longitude                                             
2010-12-31 33.49969 -85.352576  282.626894  289.846740  0.006262  0.018   
                    -85.851890  282.803184  289.971640  0.006194  0.018   
                    -85.313999  282.627318  289.836568  0.006274  0.018   
                    -85.680346  282.716456  289.929703  0.006204  0.018   
                    -85.361844  282.626793  289.849184  0.006259  0.018   
...                                    ...         ...       ...    ...   
           33.49470 -85.355252  282.631126  289.851267  0.006261  0.018   
                    -85.314852  282.631558  289.840658  0.006273  0.018   
                    -85.495289  282.629630  289.888040  0.006217  0.018   
                    -85.724953  282.742573  289.944479  0.006202  0.018   
                    -85.601858  282.680743  289.914273  0.006209  0.018   

                                duaod550    lai_hv    lai_lv            msl  \
valid_time latitude longitude                                                 
2010-12-31 33.49969 -85.352576  0.007976  3.651370  2.729497  101706.803001   
                    -85.851890  0.008337  3.528290  2.721260  101707.754081   
                    -85.313999  0.007955  3.672237  2.720927  101706.411332   
                    -85.680346  0.008202  3.549417  2.741242  101708.020108   
                    -85.361844  0.007982  3.646357  2.731556  101706.897098   
...                                  ...       ...       ...            ...   
           33.49470 -85.355252  0.007985  3.651024  2.730142  101706.792842   
                    -85.314852  0.007962  3.672853  2.721172  101706.382572   
                    -85.495289  0.008064  3.575359  2.761236  101708.214934   
                    -85.724953  0.008244  3.545817  2.735625  101707.906081   
                    -85.601858  0.008147  3.560589  2.750212  101708.101265   

                                omaod550         pm2p5  ...  aermssdul  \
valid_time latitude longitude                           ...              
2010-12-31 33.49969 -85.352576  0.077320  1.693269e-08  ...   0.000002   
                    -85.851890  0.076725  1.627143e-08  ...   0.000003   
                    -85.313999  0.077453  1.697360e-08  ...   0.000002   
                    -85.680346  0.076768  1.651758e-08  ...   0.000003   
                    -85.361844  0.077289  1.692286e-08  ...   0.000002   
...                                  ...           ...  ...        ...   
           33.49470 -85.355252  0.077294  1.692273e-08  ...   0.000002   
                    -85.314852  0.077433  1.696546e-08  ...   0.000002   
                    -85.495289  0.076814  1.677462e-08  ...   0.000003   
                    -85.724953  0.076745  1.644840e-08  ...   0.000003   
                    -85.601858  0.076774  1.662418e-08  ...   0.000003   

                                aermssbchphil  aermssomhphil  aermssbchphob  \
valid_time latitude longitude                                                 
2010-12-31 33.49969 -85.352576   1.400559e-07       0.000003   5.252117e-07   
                    -85.851890   1.345085e-07       0.000003   5.235354e-07   
                    -85.313999   1.409249e-07       0.000003   5.256258e-07   
                    -85.680346   1.355938e-07       0.000003   5.235811e-07   
                    -85.361844   1.398471e-07       0.000003   5.251122e-07   
...                                       ...            ...            ...   
           33.49470 -85.355252   1.399748e-07       0.000003   5.251665e-07   
                    -85.314852   1.408846e-07       0.000003   5.255986e-07   
                    -85.495289   1.368211e-07       0.000003   5.236686e-07   
                    -85.724953   1.352932e-07       0.000003   5.235801e-07   
                    -85.601858   1.360712e-07       0.000003   5.236010e-07   

                                aermssomhpho

In [17]:
## group by the valid time dimension. The result is same as above as there is only 1 time.
## this step is necessary to merge with the shape file

summary = year_avg_finer.groupby("valid_time").mean(["latitude", "longitude"])
summary.to_dataframe()

,d2m,t2m,bcaod550,chnk,duaod550,lai_hv,lai_lv,msl,omaod550,pm2p5,...,aermssdul,aermssbchphil,aermssomhphil,aermssbchphob,aermssomhphob,aermsssss,aermssssm,aermssssl,aermsssu,aermssso2
valid_time,,,,,,,,,,,,,,,,,,,,,
2010-12-31,282.576162,289.779605,0.006238,0.018,0.007921,3.563574,2.738629,101708.386647,0.077423,1.683235e-08,...,0.000002,1.382703e-07,0.000003,5.244256e-07,0.000013,1.999225e-07,0.000013,7.356056e-07,0.000009,0.000004


## In the next notebook, we will create a function to extract values in all counties.